In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path = "/content/drive/MyDrive/PyTorch/Data/shakespeare.txt"

with open(path, "r", encoding='utf8') as f:
    text = f.read()

In [3]:
text[:1000]

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bud buriest thy content,\n  And tender churl mak'st waste in niggarding:\n    Pity the world, or else this glutton be,\n    To eat the world's due, by the grave and thee.\n\n\n                     2\n  When forty winters shall besiege thy brow,\n  And dig deep trenches in thy beauty's field,\n  Thy youth's proud livery so gazed on now,\n  Will be a tattered weed of small worth held:  \n  Then being asked, where all thy beauty lies,\n  Where all the treasure of thy lusty days;\n  To sa

In [4]:
print(text[:1000])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:  
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lusty days;
  To say within thine own deep su

In [5]:
len(text)

5445609

# Encode Entire Text

In [6]:
all_characters = set(text)
len(all_characters)

84

In [7]:
decoder = dict(enumerate(all_characters))
decoder.items()

dict_items([(0, ':'), (1, 'V'), (2, 'k'), (3, 'w'), (4, 'C'), (5, 'H'), (6, '6'), (7, "'"), (8, 'h'), (9, ')'), (10, 'Y'), (11, 'X'), (12, '.'), (13, 'Q'), (14, '>'), (15, '?'), (16, 's'), (17, '0'), (18, 'm'), (19, '('), (20, '9'), (21, 'c'), (22, 't'), (23, 'I'), (24, '\n'), (25, 'j'), (26, '3'), (27, 'l'), (28, 'L'), (29, 'D'), (30, '&'), (31, '|'), (32, 'e'), (33, 'f'), (34, 'J'), (35, ','), (36, '4'), (37, '7'), (38, ' '), (39, 'G'), (40, '8'), (41, 'B'), (42, 'n'), (43, 'y'), (44, ';'), (45, 'A'), (46, 'E'), (47, '-'), (48, '_'), (49, 'K'), (50, 'z'), (51, 'p'), (52, 'u'), (53, 'Z'), (54, 'v'), (55, '1'), (56, '!'), (57, 'r'), (58, 'o'), (59, 'W'), (60, 'O'), (61, 'a'), (62, 'N'), (63, 'q'), (64, '<'), (65, 'P'), (66, '"'), (67, 'i'), (68, 'S'), (69, '2'), (70, 'T'), (71, 'g'), (72, 'd'), (73, '5'), (74, 'M'), (75, '['), (76, ']'), (77, 'x'), (78, 'F'), (79, 'R'), (80, 'U'), (81, 'b'), (82, '`'), (83, '}')])

In [8]:
encoder = {char : ind for ind , char in decoder.items()}
encoder.items()

dict_items([(':', 0), ('V', 1), ('k', 2), ('w', 3), ('C', 4), ('H', 5), ('6', 6), ("'", 7), ('h', 8), (')', 9), ('Y', 10), ('X', 11), ('.', 12), ('Q', 13), ('>', 14), ('?', 15), ('s', 16), ('0', 17), ('m', 18), ('(', 19), ('9', 20), ('c', 21), ('t', 22), ('I', 23), ('\n', 24), ('j', 25), ('3', 26), ('l', 27), ('L', 28), ('D', 29), ('&', 30), ('|', 31), ('e', 32), ('f', 33), ('J', 34), (',', 35), ('4', 36), ('7', 37), (' ', 38), ('G', 39), ('8', 40), ('B', 41), ('n', 42), ('y', 43), (';', 44), ('A', 45), ('E', 46), ('-', 47), ('_', 48), ('K', 49), ('z', 50), ('p', 51), ('u', 52), ('Z', 53), ('v', 54), ('1', 55), ('!', 56), ('r', 57), ('o', 58), ('W', 59), ('O', 60), ('a', 61), ('N', 62), ('q', 63), ('<', 64), ('P', 65), ('"', 66), ('i', 67), ('S', 68), ('2', 69), ('T', 70), ('g', 71), ('d', 72), ('5', 73), ('M', 74), ('[', 75), (']', 76), ('x', 77), ('F', 78), ('R', 79), ('U', 80), ('b', 81), ('`', 82), ('}', 83)])

In [9]:
encoded_text = np.array([encoder[char] for char in text])
encoded_text[:200]

array([24, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38,
       38, 38, 38, 38, 38, 55, 24, 38, 38, 78, 57, 58, 18, 38, 33, 61, 67,
       57, 32, 16, 22, 38, 21, 57, 32, 61, 22, 52, 57, 32, 16, 38,  3, 32,
       38, 72, 32, 16, 67, 57, 32, 38, 67, 42, 21, 57, 32, 61, 16, 32, 35,
       24, 38, 38, 70,  8, 61, 22, 38, 22,  8, 32, 57, 32, 81, 43, 38, 81,
       32, 61, 52, 22, 43,  7, 16, 38, 57, 58, 16, 32, 38, 18, 67, 71,  8,
       22, 38, 42, 32, 54, 32, 57, 38, 72, 67, 32, 35, 24, 38, 38, 41, 52,
       22, 38, 61, 16, 38, 22,  8, 32, 38, 57, 67, 51, 32, 57, 38, 16,  8,
       58, 52, 27, 72, 38, 81, 43, 38, 22, 67, 18, 32, 38, 72, 32, 21, 32,
       61, 16, 32, 35, 24, 38, 38,  5, 67, 16, 38, 22, 32, 42, 72, 32, 57,
       38,  8, 32, 67, 57, 38, 18, 67, 71,  8, 22, 38, 81, 32, 61, 57, 38,
        8, 67, 16, 38, 18, 32, 18, 58, 57, 43,  0, 24, 38])

# One Hot Encoding

In [10]:
test = np.array([0,1,2,3,4,5,6,7,8,9]).reshape(2,5)
test.shape , test.size

((2, 5), 10)

In [11]:
test.flatten()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [12]:
num_uni = 15

one_hot = np.zeros((test.size, num_uni))
one_hot = one_hot.astype(np.float32)

print(one_hot.shape)
print()

print(np.arange(one_hot.shape[0]))
print()

one_hot[ np.arange(one_hot.shape[0]), test.flatten()] = 1.0

print(one_hot.shape)
print()

one_hot

(10, 15)

[0 1 2 3 4 5 6 7 8 9]

(10, 15)



array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [13]:
def one_hot_encoder(encoded_text, num_uni_chars):
    '''
    encoded_text : batch of encoded text
    
    num_uni_chars = number of unique characters (len(set(text)))
    '''

    # Create a placeholder for zeros.
    # (row,columns)
    one_hot = np.zeros((encoded_text.size, num_uni_chars))

    # Convert data type for later use with pytorch (errors if we dont)
    one_hot = one_hot.astype(np.float32)

    # Using fancy indexing fill in the ls at the correct index locations
    one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()] = 1.0
    
    one_hot = one_hot.reshape( (*encoded_text.shape, num_uni_chars))

    return one_hot

In [14]:
one_hot_encoder(np.arange(10).reshape(5,2), 15).shape

(5, 2, 15)

# Creating Traning Batches

In [15]:
example_text = np.arange(10)
example_text

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [16]:
example_text.reshape((5,-1))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [17]:
def generate_batches(encoded_text , sample_per_batch = 10, seq_len = 50):

    '''
    Generate (using yield) batches for training.
    
    X: Encoded Text of length seq_len
    Y: Encoded Text shifted by one
    
    Example:
    
    X:
    
    [[1 2 3]]
    
    Y:
    
    [[ 2 3 4]]
    
    encoded_text : Complete Encoded Text to make batches from
    batch_size : Number of samples per batch
    seq_len : Length of character sequence
       
    '''

    # Total number of characters per batch
    # Example: If samp_per_batch is 2 and seq_len is 50, then 100
    # characters come out per batch.

    char_per_batch = sample_per_batch * seq_len

    # Number of batches avaiable to make
    # Use int() to roun to nearest integer
    num_batches_avail = int(len(encoded_text) / char_per_batch)

    # Cut off end of encoded_text that
    # won't fit evenly into a batch
    encoded_text = encoded_text[:num_batches_avail * char_per_batch]    

    # Reshape text into rows the size of a batch
    encoded_text = encoded_text.reshape((sample_per_batch, -1))

    # Go through each row in array.

    for n in range(0, encoded_text.shape[1], seq_len):

        x = encoded_text[:, n:n+seq_len]
        y = np.zeros_like(x)

        try:
            y[: , :-1] = x[:, 1:]
            y[: ,  -1] = encoded_text[:, n+seq_len]
        
        except:
            y[: , :-1] = x[:, 1:]
            y[: ,  -1] = encoded_text[:,0]
        
        yield x,y

In [18]:
for n in np.arange(10).reshape((5,2)):
    print(n)

[0 1]
[2 3]
[4 5]
[6 7]
[8 9]


In [19]:
sample_text = encoded_text[:100]
sample_text

array([24, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38,
       38, 38, 38, 38, 38, 55, 24, 38, 38, 78, 57, 58, 18, 38, 33, 61, 67,
       57, 32, 16, 22, 38, 21, 57, 32, 61, 22, 52, 57, 32, 16, 38,  3, 32,
       38, 72, 32, 16, 67, 57, 32, 38, 67, 42, 21, 57, 32, 61, 16, 32, 35,
       24, 38, 38, 70,  8, 61, 22, 38, 22,  8, 32, 57, 32, 81, 43, 38, 81,
       32, 61, 52, 22, 43,  7, 16, 38, 57, 58, 16, 32, 38, 18, 67])

In [20]:
sample_per_batch = 2
seq_len = 4

len(sample_text), sample_per_batch*seq_len, int(len(sample_text) / (sample_per_batch*seq_len))

(100, 8, 12)

In [21]:
12*8,len(sample_text[:12*8])

(96, 96)

In [22]:
sample_text = sample_text[:12*8]
len(sample_text)

96

In [23]:
sample_text = sample_text.reshape((sample_per_batch, -1))
sample_text.shape

(2, 48)

In [24]:
seq_len = 5
b = 0

for n in range(0, sample_text.shape[1], seq_len):

    if b == 1:
        break
    
    b+=1

    x = sample_text[:, n:n+seq_len]
    y = np.zeros_like(x)

    #print(x.shape)
    #print(y.shape)
    #print()

    try :
        y[: , :-1] = x[:, 1:]
        y[: , -1 ] = sample_text[:, n+seq_len]

    except:

        y[:, :-1] = sample_text[:, 1:]
        y[:, -1 ] = sample_text[:, 0]

    print(x)
    print()
    print(y)

[[24 38 38 38 38]
 [38  3 32 38 72]]

[[38 38 38 38 38]
 [ 3 32 38 72 32]]


In [25]:
sample_text = encoded_text[:100]

batch_generator = generate_batches(sample_text, sample_per_batch = 2, seq_len = 4)
x,y = next(batch_generator)

In [26]:
print("Shape : ", (x.shape))

print()
print("x :")
print(x)
print(x.shape)
print()
print("y :")
print(y)
print(y.shape)

Shape :  (2, 4)

x :
[[24 38 38 38]
 [38  3 32 38]]
(2, 4)

y :
[[38 38 38 38]
 [ 3 32 38 72]]
(2, 4)


In [27]:
one_hot_encoder(x, 84).shape

(2, 4, 84)

In [28]:
x.size,(*x.shape,84)

(8, (2, 4, 84))

# GPU Check

In [29]:
torch.cuda.is_available()

True

# Creating The LSTM Model

In [30]:
class CharModel(nn.Module):

    def __init__(self, all_chars, num_hidden = 256, num_layers = 4, drop_prob = 0.5, use_gpu = False):

        super().__init__()
        self.drop_prob = drop_prob
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.use_gpu = use_gpu

        self.all_chars = all_chars
        self.decoder = dict(enumerate(all_chars))
        self.encoder = {char : ind for ind,char in decoder.items()}

        self.lstm = nn.LSTM( input_size = len(self.all_chars),
                             hidden_size = num_hidden, 
                             num_layers = num_layers,
                             dropout = drop_prob,
                             batch_first = True)
        
        self.dropout = nn.Dropout(drop_prob)
        self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))

    def forward(self, x, hidden):

        lstm_output , hidden = self.lstm(x, hidden)

        drop_output = self.dropout(lstm_output)

        drop_output = drop_output.contiguous().view(-1, self.num_hidden)

        final_out = self.fc_linear(drop_output)

        return final_out,hidden

    def hidden_state(self,batch_size):

        '''
        Used as separate method to account for both GPU and CPU users.
        '''

        if self.use_gpu :

            hidden = (torch.zeros(self.num_layers, batch_size, self.num_hidden).cuda(),
                      torch.zeros(self.num_layers, batch_size, self.num_hidden).cuda())
            
        else:

            hidden = (torch.zeros(self.num_layers, batch_size, self.num_hidden),
                      torch.zeros(self.num_layers, batch_size, self.num_hidden))
            
        return hidden

In [31]:
model = CharModel(
    all_chars = all_characters,
    num_hidden = 512,
    num_layers = 3,
    drop_prob = 0.5,
    use_gpu = True
)
model

CharModel(
  (lstm): LSTM(84, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_linear): Linear(in_features=512, out_features=84, bias=True)
)

In [32]:
for name,param in model.named_parameters():
    print(name , "\t", param.shape)
    print("".center(100,"-"))

lstm.weight_ih_l0 	 torch.Size([2048, 84])
----------------------------------------------------------------------------------------------------
lstm.weight_hh_l0 	 torch.Size([2048, 512])
----------------------------------------------------------------------------------------------------
lstm.bias_ih_l0 	 torch.Size([2048])
----------------------------------------------------------------------------------------------------
lstm.bias_hh_l0 	 torch.Size([2048])
----------------------------------------------------------------------------------------------------
lstm.weight_ih_l1 	 torch.Size([2048, 512])
----------------------------------------------------------------------------------------------------
lstm.weight_hh_l1 	 torch.Size([2048, 512])
----------------------------------------------------------------------------------------------------
lstm.bias_ih_l1 	 torch.Size([2048])
----------------------------------------------------------------------------------------------------
lstm.bi

In [33]:
total_params = []
for p in model.parameters():
    print(p.numel())
    total_params.append(int(p.numel()))

print(f"_____\n{sum(total_params)}")

172032
1048576
2048
2048
1048576
1048576
2048
2048
1048576
1048576
2048
2048
43008
84
_____
5470292


In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) 

In [35]:
train_percent = 0.1

len(encoded_text)

5445609

In [36]:
train_ind = int(len(encoded_text) * (train_percent))

train_data = encoded_text[:train_ind]
val_data = encoded_text[train_ind:]

len(train_data), len(val_data)

(544560, 4901049)

# Trnaing The Network

In [37]:
epochs = 50
batch_size = 128
seq_len = 100
tracker = 0
num_char = max(encoded_text) + 1
num_char

84

In [38]:
test = model.hidden_state(batch_size)
test[0].shape, test[1].shape

(torch.Size([3, 128, 512]), torch.Size([3, 128, 512]))

In [39]:
batch = generate_batches(train_data,batch_size , seq_len) 
x,y = next(batch)

print(x)
print("-----")
print(y)

[[24 38 38 ... 38 18 67]
 [43 35 38 ... 35 24 38]
 [67 42 61 ... 16  3 32]
 ...
 [79 60 68 ... 27 61 42]
 [57 35 38 ...  3 42 32]
 [61 57 22 ... 43 24 38]]
-----
[[38 38 38 ... 18 67 71]
 [35 38 38 ... 24 38 38]
 [42 61 22 ...  3 32 32]
 ...
 [60 68 45 ... 61 42 72]
 [35 38 38 ... 42 32 57]
 [57 22 38 ... 24 38 38]]


In [40]:
model.use_gpu

True

In [41]:
epochs = 50
batch_size = 128
seq_len = 100
tracker = 0
num_char = max(encoded_text) + 1 # 84

model.train()

if model.use_gpu:
    model.cuda()

for i in range(epochs):

    # (layers, batch_size , num_hidden)
    hidden = model.hidden_state(batch_size)

    for x,y in generate_batches(train_data, batch_size , seq_len):
        
        tracker +=1
        
        # X
        # Shape : 128,100,84 

        # y 
        # Shape : 128,100
        x = one_hot_encoder(x, num_char)
        
        inputs = torch.from_numpy(x)
        targets = torch.from_numpy(y)

        if model.use_gpu:

            inputs = inputs.cuda()
            targets = targets.cuda()

        hidden = tuple([state.data for state in hidden])

        model.zero_grad()
        
        lstm_output, hidden = model.forward(inputs, hidden)
        loss = criterion(lstm_output, targets.view(batch_size*seq_len).long())
        loss.backward()

        # POSSIBLE EXPLODING GRADIENT PROBLEM!
        # LET"S CLIP JUST IN CASE
        nn.utils.clip_grad_norm_(model.parameters(), max_norm = 5)

        optimizer.step()

        ###################################
        ### CHECK ON VALIDATION SET ######
        #################################

        if tracker % 25 == 0:

            val_hidden = model.hidden_state(batch_size)
            val_losses = []
            model.eval()

            for x,y in generate_batches(val_data, batch_size , seq_len):

                x = one_hot_encoder(x, num_char)

                inputs = torch.from_numpy(x)
                targets = torch.from_numpy(y)

                if model.use_gpu:
                    
                    inputs = inputs.cuda()
                    targets = targets.cuda()

                val_hidden = tuple([state.data for state in val_hidden])

                lstm_output, val_hidden = model.forward(inputs, val_hidden)
                val_loss = criterion(lstm_output, targets.view(batch_size*seq_len).long())

                val_losses.append(val_loss.item())

            model.train()

            print(f"Epoch : {i:3}, Step : {tracker:3}, Val Loss : {val_loss.item():.5f}")

Epoch :   0, Step :  25, Val Loss : 3.23809
Epoch :   1, Step :  50, Val Loss : 3.23367
Epoch :   1, Step :  75, Val Loss : 3.22000
Epoch :   2, Step : 100, Val Loss : 3.09215
Epoch :   2, Step : 125, Val Loss : 3.01382
Epoch :   3, Step : 150, Val Loss : 2.93357
Epoch :   4, Step : 175, Val Loss : 2.79351
Epoch :   4, Step : 200, Val Loss : 2.72326
Epoch :   5, Step : 225, Val Loss : 2.62757
Epoch :   5, Step : 250, Val Loss : 2.51788
Epoch :   6, Step : 275, Val Loss : 2.43935
Epoch :   7, Step : 300, Val Loss : 2.39395
Epoch :   7, Step : 325, Val Loss : 2.36061
Epoch :   8, Step : 350, Val Loss : 2.32126
Epoch :   8, Step : 375, Val Loss : 2.33302
Epoch :   9, Step : 400, Val Loss : 2.26826
Epoch :  10, Step : 425, Val Loss : 2.24100
Epoch :  10, Step : 450, Val Loss : 2.21302
Epoch :  11, Step : 475, Val Loss : 2.18951
Epoch :  11, Step : 500, Val Loss : 2.16520
Epoch :  12, Step : 525, Val Loss : 2.14614
Epoch :  13, Step : 550, Val Loss : 2.13056
Epoch :  13, Step : 575, Val Los

In [42]:
model_name = "selfModel2.net"
torch.save(model.state_dict(), model_name)

In [43]:
model = CharModel(
    all_chars = all_characters,
    num_hidden = 512,
    num_layers = 3,
    drop_prob = 0.5,
    use_gpu = True
)
model

CharModel(
  (lstm): LSTM(84, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_linear): Linear(in_features=512, out_features=84, bias=True)
)

In [44]:
model.load_state_dict(torch.load(model_name))
model.eval()

CharModel(
  (lstm): LSTM(84, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_linear): Linear(in_features=512, out_features=84, bias=True)
)

# Generating Predictions

In [63]:
def predicted_next_char(model, char, hidden = None, k = 1):

    encoded_text = model.encoder[char]

    encoded_text = np.array([[encoded_text]])

    encoded_text = one_hot_encoder(encoded_text, len(model.all_chars))

    inputs = torch.from_numpy(encoded_text)

    if model.use_gpu:
        inputs = inputs.cuda()

    hidden = tuple([state.data for state in hidden])

    lstm_out , hidden = model(inputs, hidden)

    probs = F.softmax(lstm_out, dim = 1).data

    if model.use_gpu:
        probs = probs.cpu()

    probs, index_positions = probs.topk(k)

    index_positions = index_positions.numpy().squeeze()

    probs = probs.numpy().flatten()
    probs = probs / probs.sum()

    char = np.random.choice(index_positions, p = probs,)

    return model.decoder[char], hidden

In [64]:
x = torch.arange(1,6)
x

tensor([1, 2, 3, 4, 5])

In [65]:
probs, index_positions  = torch.topk(x,1)
index_positions = index_positions.numpy().squeeze()
index_positions,probs

(array(4), tensor([5]))

In [68]:
def generate_text(model, size , seed = "The ", k = 1):

    if model.use_gpu:
        model.cuda()
    else:
        model.cpu()

    model.eval()

    output_chars = [c for c in seed]

    hidden = model.hidden_state(1)

    for char in seed:
        char , hidden = predicted_next_char(model,char, hidden, k = k)

    output_chars.append(char)

    #return output_chars

    for i in range(size):

        char, hidden = predicted_next_char(model, output_chars[-1], hidden, k = k)

        output_chars.append(char)

    return "".join(output_chars)

In [72]:
print(generate_text(model, 5000, seed = "The ", k=3))

The that the best see that should bried,
    When I will see him still their self to see,
    Will be so more in thy self of honour,
    To be such thou thy last be so seed to thee thee thanks.
    I with this brain to be the service,
    To take him that I was the would be all.


                     11
  O that is the seases of my lov'd,
  And that I have some strange of my love,
  That think to see the way those thought thou dest,
  As true a sense than this the wings of mare,
  And serfow to the world we be so mare.
    But that is sometime on my bear and true,
    What's thy self thou art seem of my better,  
    Which is not the better best so stail and stand.


                     14
  Thou shall I shall be this thine one, were thou to be this seem,
  Which it they will be such the way to thee stand,
  The wars of time of my life they break,
  And sees the self thou that shall stain they string,
  The wint of strong thoughts they be so song of thee,
  They that is not the store